In [9]:
import torch
import torch.nn as nn
import numpy as np
import json
import os

# ==============================================================================
# 1. 모델 설계도 (그대로 유지)
# ==============================================================================
class SquatView1Model(nn.Module):
    def __init__(self):
        super(SquatView1Model, self).__init__()
        self.lstm = nn.LSTM(10, 64, num_layers=1, batch_first=True)
        self.layer_norm = nn.LayerNorm(64)
        self.fc = nn.Linear(64, 3) 
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), 64).to(x.device)
        c0 = torch.zeros(1, x.size(0), 64).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        return self.fc(self.layer_norm(out[:, -1, :]))

class SquatView3Model(nn.Module):
    def __init__(self):
        super(SquatView3Model, self).__init__()
        self.lstm = nn.LSTM(6, 128, num_layers=2, batch_first=True)
        self.layer_norm = nn.LayerNorm(128)
        self.fc = nn.Linear(128, 2) 
    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 128).to(x.device)
        c0 = torch.zeros(2, x.size(0), 128).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        return self.fc(self.layer_norm(out[:, -1, :]))

# ==============================================================================
# 2. AI 코치 (멘트 대폭 수정!)
# ==============================================================================
class SquatAnalyzer:
    def __init__(self, model_v1_path, model_v3_path):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"🔧 AI 코치 준비 중... (Device: {self.device})")

        self.model_v1 = SquatView1Model().to(self.device)
        self.model_v3 = SquatView3Model().to(self.device)

        if os.path.exists(model_v1_path):
            self.model_v1.load_state_dict(torch.load(model_v1_path, map_location=self.device))
            self.model_v1.eval() 
        else: print(f"❌ View 1 모델 없음")

        if os.path.exists(model_v3_path):
            self.model_v3.load_state_dict(torch.load(model_v3_path, map_location=self.device))
            self.model_v3.eval()
        else: print(f"❌ View 3 모델 없음")

        # 통계값
        self.v1_mean = np.array([13.5081, 119.9536, 119.0998, 156.8377, 128.0947, 144.0267, 14.4445, 9.7183, 127.5440, 162.1496], dtype=np.float32)
        self.v1_std  = np.array([9.0042, 15.7305, 9.8618, 23.7940, 24.7444, 25.4835, 8.6527, 8.9265, 10.9611, 19.4002], dtype=np.float32)
        self.v1_keys = ['trunk_inclination', 'ankle_dorsi_left', 'ankle_dorsi_right', 'hip_flexion_left', 'hip_flexion_right', 'knee_mean', 'tibia_inclination_left', 'tibia_inclination_right']

        self.v3_mean = np.array([166.6711, 166.5661, 178.8124, 178.4266, 0.8705, 0.5871], dtype=np.float32)
        self.v3_std  = np.array([16.2031, 16.3651, 5.5688, 4.2768, 0.2692, 0.2303], dtype=np.float32)
        self.v3_keys = ['fppa_left', 'fppa_right', 'pelvic_tilt', 'trunk_lateral_tilt', 'knee_width_ratio', 'head_pitch_proxy']

    def _preprocess(self, json_path, keys, mean, std, is_view1=False):
        try:
            with open(json_path, 'r', encoding='utf-8') as f: data = json.load(f)
        except: return None

        frames = []
        for frame in data['frames']:
            if 'angles' not in frame: continue
            angles = frame['angles']
            row = []
            for k in keys:
                val = angles.get(k)
                if val is None: val = 0.0
                row.append(float(val))
            
            if is_view1:
                val_l = angles.get('face_shoulder_hip_left', 180.0)
                val_r = angles.get('face_shoulder_hip_right', 180.0)
                row.append(float(val_l))
                row.append(float(val_r))
                
            frames.append(row)
            
        if not frames: return None
        if len(frames) > 64: frames = frames[:64]
        while len(frames) < 64: frames.append(frames[-1])
        
        X = (np.array(frames, dtype=np.float32) - mean) / std
        X = np.nan_to_num(X)
        return torch.tensor(X).unsqueeze(0).to(self.device)

    def run(self, json_v1, json_v3):
        print("\n" + "="*50)
        print("📊 AI 스쿼트 코칭 리포트")
        print("="*50)

        # --- View 1 ---
        print(f"📂 [측면 분석] {os.path.basename(json_v1)}")
        input_v1 = self._preprocess(json_v1, self.v1_keys, self.v1_mean, self.v1_std, is_view1=True)
        
        if input_v1 is not None:
            with torch.no_grad():
                probs = torch.sigmoid(self.model_v1(input_v1)).cpu().numpy()[0]
            
            labels = ["척추 중립", "발바닥 고정", "가동 범위"]
            for i, label in enumerate(labels):
                bad_prob = probs[i] * 100
                score = 100 - bad_prob # ★ 불량확률을 점수(100점 만점)로 변환!
                
                # 점수별 멘트 (Grade)
                if score >= 80: grade = "⭐⭐⭐ (완벽)"
                elif score >= 60: grade = "⭐⭐ (양호)"
                elif score >= 40: grade = "⚠️ (주의)"
                else: grade = "❌ (교정 필요)"
                
                print(f"  - {label:<10}: {score:.1f}점  {grade}")
                
                # 피드백 (점수 낮을 때만)
                if score < 60:
                    if i==0: print("    💡 Tip: 허리가 굽지 않게 가슴을 활짝 펴주세요.")
                    if i==1: print("    💡 Tip: 발바닥 전체로 바닥을 꾹 눌러주세요.")
                    if i==2: print("    💡 Tip: 조금만 더 깊게 앉아보세요!")
        else:
            print("  ⚠️ 데이터 오류")

        print("-" * 50)

        # --- View 3 ---
        print(f"📂 [정면 분석] {os.path.basename(json_v3)}")
        input_v3 = self._preprocess(json_v3, self.v3_keys, self.v3_mean, self.v3_std, is_view1=False)
        
        if input_v3 is not None:
            with torch.no_grad():
                probs = torch.sigmoid(self.model_v3(input_v3)).cpu().numpy()[0]
            
            # 무릎 (Index 1)
            knee_bad_prob = probs[1] * 100
            knee_score = 100 - knee_bad_prob
            
            if knee_score >= 80: k_grade = "⭐⭐⭐ (완벽)"
            elif knee_score >= 60: k_grade = "⭐⭐ (양호)"
            else: k_grade = "❌ (교정 필요)"
            
            print(f"  - 무릎 정렬   : {knee_score:.1f}점  {k_grade}")
            if knee_score < 60:
                print("    💡 Tip: 무릎이 안으로 모이지 않게 바깥으로 벌려주세요.")

            # 고개 (Index 0) - 참고용
            head_bad_prob = probs[0] * 100
            head_score = 100 - head_bad_prob
            print(f"  - 시선 처리   : {head_score:.1f}점  (참고용 지표)")
            
        else:
            print("  ⚠️ 데이터 오류")
            
        print("="*50 + "\n")

if __name__ == "__main__":
    MODEL_V1 = "squat_view1_final_model.pth"
    MODEL_V3 = "squat_view3_final_model.pth"
    
    analyzer = SquatAnalyzer(MODEL_V1, MODEL_V3)
    
    # 파일 경로
    test_json_v1 = r"D07-1-313_processed_rep00_view1.json"
    test_json_v3 = r"D07-1-313_processed_rep00_view3.json"
    
    if os.path.exists(test_json_v1) and os.path.exists(test_json_v3):
        analyzer.run(test_json_v1, test_json_v3)
    else:
        print("🚨 파일을 찾을 수 없습니다.")

🔧 AI 코치 준비 중... (Device: cpu)

📊 AI 스쿼트 코칭 리포트
📂 [측면 분석] D07-1-313_processed_rep00_view1.json
  - 척추 중립     : 80.6점  ⭐⭐⭐ (완벽)
  - 발바닥 고정    : 98.1점  ⭐⭐⭐ (완벽)
  - 가동 범위     : 47.2점  ⚠️ (주의)
    💡 Tip: 조금만 더 깊게 앉아보세요!
--------------------------------------------------
📂 [정면 분석] D07-1-313_processed_rep00_view3.json
  - 무릎 정렬   : 81.4점  ⭐⭐⭐ (완벽)
  - 시선 처리   : 72.2점  (참고용 지표)

